In [1]:
import kimports; reload(kimports); from kimports import *
import kutils; reload(kutils)

Using Theano backend.
Using gpu device 0: Tesla K80 (CNMeM is disabled, cuDNN 5103)
/home/ubuntu/anaconda2/lib/python2.7/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


<module 'kutils' from 'kutils.pyc'>

In [2]:
path = '../data/statefarm/sample'

In [9]:
model = keras.models.Sequential([
        keras.layers.normalization.BatchNormalization(input_shape=(3, 224,224)),
        keras.layers.convolutional.Convolution2D(64, 3, 3, activation='relu'),
        keras.layers.MaxPooling2D((2,2), strides=(2,2)),
        keras.layers.core.Dropout(0.5),
        keras.layers.core.Flatten(),
        keras.layers.core.Dense(200, activation='relu'),
        keras.layers.core.Dropout(0.5),
        keras.layers.core.Dense(10, activation='softmax')
    ])

In [10]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
batchnormalization_2 (BatchNormal(None, 3, 224, 224)   448         batchnormalization_input_2[0][0] 
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 64, 222, 222)  1792        batchnormalization_2[0][0]       
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 64, 111, 111)  0           convolution2d_2[0][0]            
____________________________________________________________________________________________________
dropout_3 (Dropout)              (None, 64, 111, 111)  0           maxpooling2d_1[0][0]             
___________________________________________________________________________________________

In [20]:
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

In [14]:
train_batches = keras.preprocessing.image.ImageDataGenerator(
    ).flow_from_directory(path+'/train', target_size=(224,224), batch_size=128)
valid_batches = keras.preprocessing.image.ImageDataGenerator(
    ).flow_from_directory(path+'/valid', target_size=(224,224), batch_size=128)

Found 1000 images belonging to 10 classes.
Found 1000 images belonging to 10 classes.


In [ ]:
model.fit_generator(
    train_batches,
    train_batches.nb_sample,
    10,
    validation_data=valid_batches,
    nb_val_samples=valid_batches.nb_sample)

Epoch 1/10
 872/1000 [=========================>....] - ETA: 1s - loss: 0.1582 - acc: 0.9771

In [102]:
cl = model.layers[1]

In [46]:
r = np.matrix([[-1,-1,-1], [1,1,1], [0,0,0]])

In [103]:
unrolled = []
for convw in cl.get_weights()[0]:
    for chanw in convw:
        unrolled += [chanw, np.rot90(chanw, 1), np.rot90(chanw, 2), np.rot90(chanw, 3)]

In [104]:
ssq = map(lambda (i, u): (i, np.sum(np.power(r - u/np.mean(np.abs(u)), 2))), enumerate(unrolled))

In [105]:
sorted(ssq, key=lambda (i,u): u)

[(520, 3.2758860496811169),
 (369, 3.6200762754439282),
 (294, 3.7023243039781706),
 (704, 4.7638441690529767),
 (449, 5.493843567919626),
 (391, 5.6637741671572641),
 (307, 5.6754507806378953),
 (451, 5.7297589708518544),
 (292, 6.075461837642874),
 (595, 6.1626182699450105),
 (698, 6.2226048968133014),
 (87, 6.2306319194715876),
 (522, 6.3519503382972973),
 (22, 6.6786096796753824),
 (4, 6.9520523505542151),
 (258, 7.0123297318714428),
 (82, 7.0537609849979637),
 (413, 7.2608371247075052),
 (256, 7.3099062785404492),
 (632, 7.4446826584066166),
 (97, 7.4558013319696954),
 (570, 7.4647575092377068),
 (33, 7.582703421936511),
 (699, 7.6398515961464923),
 (51, 7.6705252729069544),
 (477, 7.7092015567581811),
 (434, 7.8216195425014075),
 (297, 7.8519718686218631),
 (325, 7.8733062771238984),
 (611, 7.9185139832980873),
 (85, 7.9946938472669977),
 (432, 8.0030052980403479),
 (609, 8.1796035802331755),
 (35, 8.1896660878519985),
 (552, 8.2534372877322042),
 (573, 8.3405270582115918),
 (634

In [106]:
unrolled[520]

array([[-0.04192427, -0.05300841, -0.08524188],
       [ 0.03587552,  0.04599271,  0.03585123],
       [-0.01905144, -0.05919175, -0.03215697]], dtype=float32)

In [107]:
unrolled[520]/np.mean(np.abs(unrolled[520]))

array([[-0.92413372, -1.16846073, -1.87898088],
       [ 0.79080164,  1.01381397,  0.79026616],
       [-0.41994962, -1.30475962, -0.70883393]], dtype=float32)

In [83]:
np.multiply(1/np.mean(unrolled[73]), unrolled[73])

array([[ 1.28942895,  2.7069509 ,  5.14823055],
       [-4.16084623, -2.19202805, -8.0152998 ],
       [ 3.56173849,  3.47819924,  7.18362665]], dtype=float32)